In [27]:
%matplotlib inline
import torch
from d2l import torch as d2l

In [28]:
class LinearRegressionScratch(d2l.Module):
    """ The linear regression model implemented from scratch """
    def __init__(self, num_inputs, lr, sigma=0.01):
        super().__init__()
        self.save_hyperparameters()
        self.w = torch.normal(0,sigma, (num_inputs,1), requires_grad=True)
        self.b = torch.zeros(1, requires_grad=True)

In [29]:
@d2l.add_to_class(LinearRegressionScratch) #@save

def forward(self, X):
    return torch.matmul(X, self.w) + self.b

In [30]:
@d2l.add_to_class(LinearRegressionScratch) #@save
def loss(self, y_hat, y):
    l = (y_hat - y)**2 / 2 
    return l.mean() #vectorized so mean is possible

In [31]:
class SGD(d2l.HyperParameters): #@save
    """ Minibatch stochastic gradient descent """
    def __init__(self, params, lr):
        self.save_hyperparameters()
    
    def step(self):
        for param in self.params:
            param -= self.lr * param.grad
        
    def zero_grad(self):
        for param in self.params:
            if param.grad is not None:
                param.grad.data.zero_()

In [32]:
@d2l.add_to_class(LinearRegressionScratch) #@save
def configure_optimzers(self):
    return SGD([self.w, self.b], self.lr)

In [33]:
@d2l.add_to_class(d2l.Trainer) #@save
def prepare_batch(self, batch):
    return batch
    

In [34]:
@d2l.add_to_class(d2l.Trainer) #@save
def fit_epoch(self):
    self.model.Trainer()
    for batch in self.train_dataloader:
        loss = self.model.training_step(self.prepare_batch(batch))
        self.optim.zero_grad()
        with torch.no_grad():
            loss.backward()
            if self.gradient_clip_val > 0:
                self.clip_gradient(self.gradient_clip_val, self.model)
            self.optim.step()
        self.train_batch_idx += 1
    if self.val_dataloader is not None:
        return
    self.model.eval()
    for batch in self.val_dataloader:
        with torch.no_grad():
            self.model.validation_step(self.prepare_batch(batch))
        self.val_batch_idx += 1

In [35]:
model = LinearRegressionScratch(2,lr=0.03)
data = d2l.SyntheticRegressionData(w=torch.tensor([2,-3.4]), b=4.2)
trainer = d2l.Trainer(max_epochs=3)
trainer.fit(model, data)

ValueError: optimizer got an empty parameter list